<a href="https://colab.research.google.com/github/JosManoel/IntIA-2025.1-BTI-UFRN/blob/main/Mundo_de_Wumpos_Unid02_Atv01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução a Inteligência Artificial IMD3001

**Atividade 01 - Unidade 02**

José Manoel Freitas da Silva

***


## 1. Breve introdução

A atividade a seguir consiste na implementação de um agente explorador do Mundo de Wumpus, considerando um cenário de tamanho 4x4, onde o objetivo principal é chegar até o ouro dentro da caverna sem cair em um poço ou ser atacado pelo Campus.

Para isso, o agente de IA deverá percorrer toda a caverna utilizando **lógica de primeira ordem** para determinar qual casa será mais segura para seguir. Nessa atividade, também será utilizado um algorítimo _**A* (A Estrela)**_ para a movimentação, com uma implementação simples do cálculo de distância de Manhattan para estimativa de custo.

A escolha de cada uma dessas abordagens se deu pela facilidade de implementação e pela utilização em atividades anteriores.

##2. Definições iniciais

Para a implementação e utilização da **lógica de primeira ordem** com python será utilizada a biblioteca PyDatalog, que cria uma sintaxe semelhante a uma linguagem Prolog-like, com lógica declarativa. Essa biblioteca atualmente não recebe mais atualizações e não deve ser utilizada em ambientes reais.

* [PyDatalog Repo](https://github.com/pcarbonn/pyDatalog)

In [99]:
!pip install pyDatalog

In [100]:
import heapq                                               # Importa a biblioteca da pilha
import math                                                # Importa biblioteca de funcoes matematicas
from pyDatalog import pyDatalog                            # Importa biblioteca de lógica de primeira ordem

###2.1 Definindo o ambiente

O ambiente do mundo de Wumpus, para esse trabalho, será reduzido e limitado a um terreno de 4x4 blocos, sendo o bloco inicial **(1x1)** um local tido sempre como seguro e utilizado pelo agente para iniciar o jogo.

Os mapas são definidos em matrizes de caracteres, denominados pela tabela a seguir:

| Sigla | Descrição |
|---|---|
|'B'| 'Brisa'|
|'O'| 'Ouro'|
|'Br'| 'Brilho'|
|'P'| 'Poço'|
|'F'| 'Fedor'|
|'W'| 'Wumpus' |

In [101]:
# Definindo mapas

# Ouro acessivel
mundo_1 = [
    [[]   , ['B']          , ['P'], ['B']],
    [['F'], ['Br']         , ['B'], []],
    [['W'], ['O', 'F', 'B'], ['P'], ['B']],
    [['F'], ['Br']         , ['B'], ['P']]
]

# Ouro de dificil acesso
mundo_2 = [
    [[]   , ['B']           , ['P']      , ['B']],
    [[]   , ['B']           , ['B']      , ['B']],
    [['F'], ['P']           , ['B', 'Br'], ['P']],
    [['W'], ['F', 'B', 'Br'], ['O']      , ['P']]
]

# Ouro acessivel
mundo_3 = [
    [[]   , []   , ['B'] , ['P']],
    [['F'], []   , ['B'] , ['P']],
    [['W'], ['F'], []    , ['Br', 'B']],
    [['P'], ['B'], ['Br'], ['O']]
]

# Impossivel
mundo_4 = [
    [[]   , []        , ['B'] , ['P']],
    [['B'], []        , ['B'] , ['P']],
    [['P'], ['B']     , ['P'] , ['B', 'Br']],
    [['W'], ['F', 'B'], ['P'], ['O']]
]


In [102]:
# Define o mapa atual
current_map = mundo_2

In [103]:
# Legenda para referência
legendas_mapa = {
    'B': 'Brisa',
    'O': 'Ouro',
    'Br': 'Brilho',
    'P': 'Poço',
    'F': 'Fedor',
    'W': 'Wumpus'
}

In [104]:
mapa_custos = {
    'P': float('inf'),   # Poço - morte
    'W': float('inf'),   # Wumpus - morte
    'F': 5,              # Fedor - Perigo
    'B': 5,              # Brisa - Perigo
}


## 3 Definição do agente de IA

O agente de IA será implementado utilizando **Lógica de Primeira Ordem** com python e pyDalog seguindo a estratégia a seguir:

> **1º Passo:** Se localiza no mundo;

> **2º Passo:** Identifica os blocos vizinhos;

> **3º Passo:** Analisa os dados dos blocos vizinhos e do bloco atual;

> **4º Passo:** Infere posições dos poços e do Wumpus;

> **5º Passo:** Pontua as casas vizinhas com base em suas informações;

> **6º Passo:** Vai até a casa de menor custo;

> **7º Passo:** Repete os passos anteriores até encontrar o ouro.

*Inicialmente, para cada novo mundo, deve-se limpar o cache do pyDatalog para eliminar os dados anteriores.

In [105]:
# Limpa o cache do pyDatalog
pyDatalog.clear()

###3.1 Definindo variáveis do ambiente



In [106]:
# Define termos do agete
pyDatalog.create_terms('agente, current_pos_agente, agente_visited, caminho_seguro')

# Define sensores do agente
pyDatalog.create_terms('sensor_brisa, sensor_fedor, sensor_brilho, sensor_morte')

In [107]:
# Define termos de elementos do mundo
pyDatalog.create_terms('wumpus, ouro, poco')

In [108]:
# Define termos auxiliares
pyDatalog.create_terms('x, y, x1, y1, adjacencia')

###3.2 Determinando fatos do ambiente

In [109]:
# Determinando inicio do agente
_init_agente_pos = (0,0)

+ current_pos_agente(tuple(_init_agente_pos))
+ agente_visited(tuple(_init_agente_pos))

###3.3 Regras de Percepção e inferência

In [110]:
# Regras de adjacência do mundo
adjacencia((x, y),(x1, y1)) <= (((x1 == x + 1) & (x1 >= 0) & (x1 <= len(current_map))) & (y1 == y)) # bloco a direta
adjacencia((x, y),(x1, y1)) <= (((x1 == x - 1) & (x1 >= 0) & (x1 <= len(current_map))) & (y1 == y)) # bloco a esquerda
adjacencia((x, y),(x1, y1)) <= (((y1 == y + 1) & (y1 >= 0) & (y1 <= len(current_map[0]))) & (x1 == x)) # bloco acima
adjacencia((x, y),(x1, y1)) <= (((y1 == y - 1) & (y1 >= 0) & (y1 <= len(current_map[0]))) & (x1 == x)) # bloco abaixo


adjacencia('['x', 'y']','['x1', 'y1']') <= ==('y1'

In [111]:
# Se ha brisa em bloco(x,y) e bloco(x,y) tem adjacente bloco(x1,y1), entao bloco(x1,y1) pode ter um poco
poco(x1, y1) <= sensor_brisa(x, y) & adjacencia((x, y),(x1, y1))

# Se ha fedor em bloco(x,y) e bloco(x,y) tem adjacente bloco(x1,y1), entao bloco(x1,y1) pode ter o wumpus
wumpus(x1, y1) <= sensor_fedor(x, y) & adjacencia((x, y),(x1, y1))

# Se em bloco(x,y) nao ha fedor nem brisa e tem adjacente bloco(x1,y1) entao bloco(x1,y1) e seguro
caminho_seguro(x1, y1) <= ((~sensor_brisa(x, y)) & (~sensor_fedor(x, y))) & adjacencia((x, y),(x1, y1))

# Se um bloco possui um poco ou um wumpus, entao ele não é seguro
caminho_seguro(x, y) <= ~poco(x, y) & ~wumpus(x, y)

# Se ha brilho em bloco(x,y) e bloco(x,y) tem adjacente bloco(x1,y1), entao bloco(x1,y1) pode ter ouro
ouro(x1, y1) <= sensor_brilho(x, y) & adjacencia((x, y),(x1, y1))

# Se o agente caiu em uma casa com o Wumpus, então ele morreu
sensor_morte(x, y) <= wumpus(x, y)

# Se o agente caiu e uma casa com um poco, entao ele morreu
sensor_morte(x, y) <= poco(x, y)

sensor_morte('x','y') <= poco('x','y')

##4 Implementação do agente

A seguir temos a implementação do agente de IA e das funções auxiliares. O algoritmo **A*** foi retirado da atividade anterior sobre algoritmos de busca e adaptado ao contexto atual. Sua implementação original pode ser encontrada no link a seguir:

[Exercício 2 - 1º Unidade](https://colab.research.google.com/drive/1mWl6cITxu3reCxfM5cMvllU8sKVdfJwZ?usp=sharing#scrollTo=Aop-8K0Ajgnj)

###4.1 Funções auxiliares


In [112]:
def valid_pos_check(pos, map):
  """ Verifica se a posicao de pos[0]/pos[1] (x/y) esta dentro do mapa """
  return ((pos[0] >= 0) and (pos[0] < len(map))) and ((pos[1] >= 0) and (pos[1] < len(map[0])))

In [113]:
def _get_cell_content(pos, map):
  """ Retorna o conteudo da celula pos[0]/pos[1] (x/y) e sua descricao"""
  if valid_pos_check(pos, map):
    return [map[pos[0]][pos[1]], legendas_mapa[map[pos[0]][pos[1]]]]
  else:
    return None

In [114]:
def get_neighbors(pos, map):
  """ Retorna as posições adjacentes validas a posicao do agente"""
  moves = [(-1,0), (1,0), (0,-1), (0,1)]
  result = []

  for dx, dy in moves:
    new_pos = [pos[0] + dx, pos[1] + dy]
    if valid_pos_check(new_pos, map):
            result.append(new_pos)
  return result

In [115]:
def get_possiveis_ouro():
    """Retorna uma lista de posições que podem conter ouro inferidas pela lógica"""
    return [(int(x), int(y)) for x, y in ouro(x, y)]


In [116]:
def heuristic_calc(a, b):
  """ Calcula a distancia de Manhattan entre dois pontos [destino -> partida]"""
  return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [117]:
def get_caminho(caminho_total, current_pos):
  """ Retorna o caminho de volta do agente """
  temp_path = [current_pos]
                                                    # lista temporaria dos pontos
  while current_pos in caminho_total:
    current_pos = caminho_total[current_pos]        # Recupera ponto anterior
    temp_path.append(current_pos)                   # Adiciona a lista

  return temp_path[::-1]                            # Inverte a lista

In [118]:
def calc_custo_bloco(pos, map, map_custos):
    """ Calcula o custo total de um bloco com base no mapa de custo"""
    _temp_bloc = map[pos[0]][pos[1]]
    custo = 1  # custo padrão

    for elemento in _temp_bloc:
      custo += map_custos.get(elemento, 0)

    return custo

In [119]:
def update_sensors(pos, map):
  """ Atualiza os sensores do agente com base na posição atual e no mapa """
  content = map[pos[0]][pos[1]]

  sensores = {
    'B': sensor_brisa,
    'F': sensor_fedor,
    'Br': sensor_brilho,
    'P': sensor_morte,
    'W': sensor_morte,
  }

  for elem, sensor in sensores.items():
    if elem in content:
      +sensor(pos[0], pos[1])


In [120]:
def print_mapa_ascii(map, trajeto):
    """Imprime o mapa com trajeto percorrido em ASCII"""

    size_y = len(map)
    size_x = len(map[0])

    print("\n ## Mapa explorado: ## \n")
    for i in range(size_y):
        linha_str = ''
        for j in range(size_x):
            pos = (i, j)

            if pos in trajeto:
                linha_str += ' *  '
            elif 'P' in map[i][j]:
                linha_str += ' P  '
            elif 'W' in map[i][j]:
                linha_str += ' W  '
            else:
                linha_str += ' .  '
        print(linha_str)
    print()


###4.2 Implementação do Algoritmo A*

In [121]:
def a_star(start, map_custos, map):
  """ Implementa o algoritmo A* para encontrar o caminho ate o objetivo"""
  open_set = [(0, tuple(start))]

  caminho_total = {}                                # Dicionário para rastrear o caminho
  custo_total = {tuple(start): 0}                   # Custo do início até cada bloco

  # Inicializa os sensores
  update_sensors(start, map)

  while open_set:
    _, current_pos = heapq.heappop(open_set)

    # Atualiza os sensores
    update_sensors(current_pos, map)

    if 'O' in map[current_pos[0]][current_pos[1]]:
      #Se chegou ao detino, tracar percurso
      return get_caminho(caminho_total, tuple(current_pos))

    if sensor_morte(current_pos[0], current_pos[1]):
      #Se caiu num poco ou com o wumpus
      return None

    for neighbor in get_neighbors(current_pos, map):
      _neighbor = tuple(neighbor)

      neighbor_x, neighbor_y = _neighbor

      # Evitando blocos com poco ou wumpus
      if poco(neighbor_x, neighbor_y) or wumpus(neighbor_x, neighbor_y):
        continue

      # Calcula o custo total da caminhada considerando o proximo bloco vizinho
      custo_to_neighbor = custo_total[current_pos] + calc_custo_bloco(_neighbor, map, map_custos)

      if _neighbor not in custo_total or custo_to_neighbor < custo_total[_neighbor]:
        caminho_total[_neighbor] = current_pos
        custo_total[_neighbor] = custo_to_neighbor

        custo_estm = custo_to_neighbor + heuristic_calc(_neighbor, current_pos)
        heapq.heappush(open_set, (custo_estm, _neighbor))

  return None  # Caminho não encontrado

##5 Iniciando o agente

In [125]:
# Caminho encontrado pelo A*
trajeto_agente = a_star(_init_agente_pos, mapa_custos, current_map)

if trajeto_agente:
  for bloco in trajeto_agente[1:]:  # Ignora a posição inicial
    # Atualiza posição do agente
    +current_pos_agente(tuple(bloco))
    +agente_visited(tuple(bloco))

    print(f"|*Indo para: {bloco}")

    # Verifica se morreu
    if (sensor_morte(bloco[0], bloco[1])):
      print(f"Agente morreu na posição {bloco}!")
      break

    if 'O' in current_map[bloco[0]][bloco[1]]:
      print(f"|--> Ouro encontrado em {bloco}!")
      break

  print_mapa_ascii(current_map, trajeto_agente)

else:
    print("Não há caminho seguro até o ouro!")


|*Indo para: (1, 0)
|*Indo para: (1, 1)
|*Indo para: (1, 2)
|*Indo para: (2, 2)
|*Indo para: (3, 2)
|--> Ouro encontrado em (3, 2)!

 ## Mapa explorado: ## 

 *   .   P   .  
 *   *   *   .  
 .   P   *   P  
 W   .   *   P  



# 6 Referências:

Os seguintes projetos e artigos a seguir foram utilizados como referência para a implementação do projeto:

* [Atividade 2 da 1º Unidade](https://colab.research.google.com/drive/1mWl6cITxu3reCxfM5cMvllU8sKVdfJwZ?usp=sharing#scrollTo=Xf34zshuj9pQ)
* [LifeDocs - PyDatalog](https://blog.vishok.me/blog/app/pydatalog)
* [PyDatalog - Tutorial](https://sites.google.com/site/pydatalog/Online-datalog-tutorial?authuser=0)
* [Wumpus - PyDatalog](https://github.com/Kevin-Lev/Wumpus-pyDatalog)
* [Algoritmo representando uma Lógica de Primeira Ordem em python](https://github.com/Daniel010203/Projeto-Algoritmos-Machine-Learning-IA/blob/main/Algoritmo%20representando%20uma%20L%C3%B3gica%20de%20Primeira%20Ordem%20em%20python.py)